In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from pandas import Series, DataFrame, Panel
import matplotlib.pyplot as plt
from matplotlib import collections as mc 
import os

import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

# Tunable Variables
days_to_process = 7
pd.options.display.max_rows = 1000

# Loading data
col_names = ['TrackID','FrameNo', 'X', 'Y']
day1 = pd.read_table("001_20130101.txt",delimiter=" ", header=None, names=col_names)
day2 = pd.read_table("001_20130102.txt",delimiter=" ", header=None, names=col_names)
day3 = pd.read_table("001_20130103.txt",delimiter=" ", header=None, names=col_names)
day4 = pd.read_table("001_20130104.txt",delimiter=" ", header=None, names=col_names)
day5 = pd.read_table("001_20130105.txt",delimiter=" ", header=None, names=col_names)
day6 = pd.read_table("001_20130106.txt",delimiter=" ", header=None, names=col_names)
day7 = pd.read_table("001_20130107.txt",delimiter=" ", header=None, names=col_names)

frames = [day1, day2, day3, day4, day5, day6, day7]
concatDay = pd.concat(frames)
concatDay = concatDay.sort_values(by='TrackID', ascending=True)
concatDay



,TrackID,FrameNo,X,Y
21,6,28,166,268
30,6,37,177,262
29,6,36,177,263
28,6,35,177,263
27,6,34,176,264
26,6,33,174,264
11,6,18,142,282
10,6,17,140,283
9,6,16,137,284
8,6,15,134,287


In [8]:
# Prepare .tra extension file for TraClus clustering algorithm
outputFile = []
concatDay = concatDay[['TrackID', 'X', 'Y']]
concatDayWithoutID = concatDay[['X', 'Y']]
concatDayWithID = concatDay[['TrackID']]
concatDayWithID = concatDayWithID.drop_duplicates('TrackID')
concatDayWithID = concatDayWithID.values  #it's an array now


# slicing the dataframe based on ID
row_iterator = concatDay.iterrows()
row_id, this_row = next(row_iterator)
#slicing dataframe to respective arrays for LineCollection function
initialID = this_row['TrackID']
slicingArray = []

####################suspected bug################################
for i, rows in row_iterator:
    if rows['TrackID'] != initialID:
        slicingArray.append(i)
        initialID = rows['TrackID']
        
newList = np.split( concatDayWithoutID, slicingArray )
####################suspected bug################################

#Now: We need to combine every single piece of data to conform with .tra
#header: number of dimensions
#      : number of trajectories
number_of_dimension = 2
number_of_trajectory = len(concatDayWithID)
arrayForHeader = str(number_of_dimension) + '\n' + str(number_of_trajectory) + '\n'

#format: TrackID no_trajectory_point X1 Y1 X2 Y2... Xn Yn
arrayForXY = []
number_of_ID = len(concatDayWithID)
counter_for_points = 0
counter_for_ID = 0
# Probably need to create another outer loop for diff sets of newList, like newList[1], newList[2]

while counter_for_ID < number_of_ID:
    # reset
    counter_for_points = 0
    # Convert ID to proper string form
    tempID = concatDayWithID[counter_for_ID]
    number_of_trajectory_points = len(newList[counter_for_ID])
#     print(number_of_trajectory_points)
    stringForTempID = str(tempID)
    stringForTempID = stringForTempID[1:-1]
    # concatenation
    if len(arrayForXY) == 0:
        arrayForXY = stringForTempID + ' ' + str(number_of_trajectory_points) + ' '
    else:
        arrayForXY += stringForTempID + ' ' + str(number_of_trajectory_points) + ' '
    while counter_for_points < number_of_trajectory_points:  
        # Select X and Y
        tempHelper = newList[counter_for_ID]
        tempArray = tempHelper.iloc[counter_for_points]
        helperX = tempArray[0]
        helperY = tempArray[1]
        arrayForXY += str(helperX) + ' ' + str(helperY) + ' '
        counter_for_points += 1
        if counter_for_points == number_of_trajectory_points:
            arrayForXY += '\n'
    counter_for_ID += 1

finalOutput = arrayForHeader + arrayForXY    

# print(newList[0])
# print(tempArray)
# print(arrayForHeader)
# print(arrayForXY)
# print(len(arrayForXY))
# print((concatDayWithID))
# print(finalOutput)

# output = open('LOST2.tra', 'w')
# output.write(finalOutput)
# output.close()
# print(newList)



      TrackID    X    Y
21          6  166  268
30          6  177  262
29          6  177  263
28          6  177  263
27          6  176  264
26          6  174  264
11          6  142  282
10          6  140  283
9           6  137  284
8           6  134  287
7           6  134  287
6           6  128  289
5           6  130  289
4           6  134  285
3           6  132  286
2           6  135  289
1           6  132  286
0           6  126  286
22          6  169  267
23          6  169  265
31          6  183  260
24          6  172  265
32          6  179  261
34          6  180  261
19          6  163  270
18          6  161  270
17          6  158  273
16          6  155  273
15          6  152  277
14          6  150  278
13          6  147  277
12          6  146  280
20          6  165  268
25          6  174  264
36          6  180  258
35          6  180  257
33          6  179  258
44          9  307  255
43          9  292  253
42          9  277  253
41          9  2

In [238]:
# Get the output from TraClus, revamp and prepare data to visualize
# to achieve this, i need to read from text file and reconstruct it back to a dataframe.
fileOpener = open('LOSTResult.txt', 'r')
lines_ori = fileOpener.readlines()
# print(type(lines))

#Choose the rows with X Y coordinates
start = 2
end = len(lines_ori) 
step = 2
lines = lines_ori[start:end:step]
# print(lines)

#Choose the rows with ID and number of points
clusterID_array = []
num_points = []
start = 1
end = len(lines_ori)
step = 2
lines_ID = lines_ori[start:end:step]
length_lines_ID = len(lines_ID)
counter_for_linesID = 0
print(lines_ID[0])
while counter_for_linesID < length_lines_ID:
    temp_lines_ID = str(lines_ID[counter_for_linesID])
    temp_index = temp_lines_ID.find(" ")
    temp_index2 = temp_lines_ID.find("  ", 14, len(temp_lines_ID))
    clusterID_array += [temp_lines_ID[temp_index + 1]]
    num_points += [temp_lines_ID[temp_index2 + 2]]
    counter_for_linesID += 1
    
#Strip the string into appropriate form
element = str(lines)
elementText = element[0:-1]
elementText = str.replace(elementText, '\\' , '')
elementText = str.replace(elementText, "n", '')
elementText = str.replace(elementText, "   ", " ")
elementText = str.replace(elementText, ",", "\n")
elementText = str.replace(elementText, "\n", '')
elementText = str.replace(elementText, "'", "" )
elementText = str.replace(elementText, "  "," ")
elementText = str.replace(elementText, "[", "")
print(type(elementText))
print(elementText)

#put in 2d array form and convert to integer
numpyString = np.fromstring(elementText, dtype=float, sep= ' ' or '\n')
numpyInt = numpyString.astype(int)
number_of_rows = int( len(numpyInt)/2 )
numpyInt = numpyInt.reshape((number_of_rows,2))

#convert numpy array to dataframe
dataframeInt = pd.DataFrame(numpyInt)
dataframeInt.columns = ['X', 'Y']
dataframeInt = dataframeInt.convert_objects(convert_numeric=True)

#Assign ID to specific tracks
temp_array_ID = []
total_num_points = len(num_points)
counter_for_total_num = 0
while counter_for_total_num < total_num_points:
    tempCounter_for_num = 0
    while tempCounter_for_num < int(num_points[counter_for_total_num]):
        temp_array_ID += [clusterID_array[counter_for_total_num]]
        tempCounter_for_num += 1
    counter_for_total_num += 1

#finalized output
dataframeInt['TrackID'] = temp_array_ID
cols = dataframeInt.columns.tolist()
cols = cols[-1:] + cols[:-1]
dataframeInt = dataframeInt[cols]
    
# print(numpyInt)
# print(dataframeInt.dtypes)
print(dataframeInt)
print(type(numpyInt))

# dummyTest2 = list(numpyInt)
# dummyTest2 = dataframeInt
# print(dummyTest2)

clusterID: 0  Points Number:  6

<class 'str'>
196.41450344927262 255.3794492270473 162.6377467166942 267.48083782294344 129.8989825217174 282.53453471741153 98.0190474700542 298.9151008920112 66.19876309741849 315.3001234027702 12.057723439844963 340.997302112209 180.99940848595156 246.15024456830858 229.98922806637634 248.7360711404333 299.9790481997296 253.32175726870193 344.01741595397795 256.5763476896077 381.002022643583 259.48624852994925 421.01066467618045 262.29117225014215 463.00550018152205 265.6029538934679 
   TrackID    X    Y
0        0  196  255
1        0  162  267
2        0  129  282
3        0   98  298
4        0   66  315
5        0   12  340
6        1  180  246
7        1  229  248
8        1  299  253
9        1  344  256
10       1  381  259
11       1  421  262
12       1  463  265
<class 'numpy.ndarray'>


C:\Users\Ice\Anaconda3\lib\site-packages\ipykernel\__main__.py:55: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [239]:
# Construct lines for visualization in plot
# day1 = day1[['TrackID','X','Y']]
# dfPlotDay1 = day1[['X', 'Y']]
day1 = dataframeInt[['TrackID','X','Y']]
dfPlotDay1 = dataframeInt[['X', 'Y']]

row_iterator = day1.iterrows()
row_id, this_row = next(row_iterator)
#slicing dataframe to respective arrays for LineCollection function
initialID = this_row['TrackID']
slicingArray = []
for i, rows in row_iterator:
    if rows['TrackID'] != initialID:
        slicingArray.append(i)
        initialID = rows['TrackID']

testChamp = np.split( dfPlotDay1, slicingArray )
testChampSize = len(testChamp)

###commented because not workable####
# indices = [0, 1]
# abc = np.take(testChamp[1], indices, axis = 1)
# abc = abc.as_matrix()
#####################################
dummyTest2 = list(testChamp)

print(testChamp)

[      X    Y
0    95  290
1    99  284
2   100  288
3    98  290
4    95  293
5    90  296
6    85  298
7    81  301
8    75  302
9    69  305
10   62  308
11   56  309
12   51  312
13   43  315
14   38  317
15   32  320
16   25  323
17   18  326
18   12  328
19    7  334
20    3  335,       X    Y
21  450  263
22  445  264
23  428  261
24  407  260
25  389  258
26  373  256
27  361  254
28  349  255
29  335  251
30  316  251
31  303  250
32  292  250
33  268  248
34  259  247
35  244  247
36  230  246,       X    Y
37  485  267
38  483  268
39  480  266
40  475  265
41  471  265
42  464  264
43  458  264
44  450  263
45  442  263
46  436  262
47  424  263
48  419  262
49  409  261
50  402  260
51  393  260
52  388  258
53  381  258
54  375  258
55  367  257
56  364  257
57  353  255
58  349  255
59  340  255
60  336  255
61  327  254
62  322  254
63  316  253
64  319  253,       X    Y
65  381  260
66  377  257
67  366  257
68  358  255
69  347  255
70  328  253
71  317  252
72  302 

In [ ]:
#Clustering lines
# indices = [1, 2]
indices = [0, 1]
counterForLineConstruction = 0

#line preparation
linesegment = mc.LineCollection(dummyTest2, linewidths = 2, linestyles='solid', colors='black')
# linesegment = mc.LineCollection(dataframeInt, linewidths = 2, linestyles='solid', colors='black')

#canvas setup
x = np.arange(641)
ys = x[0:480, np.newaxis]

# set plot limits
ax = plt.axes()
ax.set_xlim(x.min(), x.max())
ax.set_ylim(ys.min(), ys.max())
ax.add_collection(linesegment)
plt.gca().invert_yaxis()
im = plt.imread('overlayingImage.png')
implot = plt.imshow(im)
plt.show()

In [2]:
#Remake data for clustering purposes
#Clustering tracks for day1
day1 = day1[['X','Y']]
day2 = day2[['X','Y']]
day3 = day3[['X','Y']]
day4 = day4[['X','Y']]
day5 = day5[['X','Y']]
day6 = day6[['X','Y']]
day7 = day7[['X','Y']]

# concatDay = [day1, day2, day3, day4, day5, day6, day7]
concatDayTrain = [day1, day2, day3, day4, day5]
concatDayTrain = pd.concat(concatDayTrain)
concatDayTrain

concatDayTest = [day6, day7]
concatDayTest = pd.concat(concatDayTest)
concatDayTest

kmeans = KMeans(n_clusters=8, random_state=0).fit(concatDayTrain)
kmeansTrans = kmeans.transform(concatDayTrain)
centroids = kmeans.cluster_centers_
distance = kmeans.inertia_
labels = kmeans.labels_    #indication for each datapoint that determines the cluster it belongs to

tempDF = concatDayTrain
tempDF['cluster_label'] = Series(labels, index = tempDF.index)

#color code assignment
tempColor = np.where(tempDF['cluster_label'] == 0, 'black', 
                     np.where( tempDF['cluster_label'] == 1, 'yellow', 
                              np.where(tempDF['cluster_label'] == 2, 'magenta', 
                                       np.where(tempDF['cluster_label'] == 3, 'cyan', 
                                                np.where(tempDF['cluster_label'] == 4, 'red', 
                                                         np.where(tempDF['cluster_label'] == 5, 'green', 
                                                                 np.where(tempDF['cluster_label'] == 6, 'blue', 'orange'))))) ))


centroidsDF = pd.DataFrame(data=centroids[0:,0:], columns=['X','Y'])
# plt.scatter(centroidsDF['X'], centroidsDF['Y'], c='black', linewidths=50, marker='x')
# plt.scatter(concatDayTrain['X'], concatDayTrain['Y'], c=tempColor, s= 50) #red color dots as training set


print(kmeansTrans.size)
print(concatDayTrain.size)

# prediction
prediction = kmeans.predict(concatDayTest)
tempDF = concatDayTest
tempDF['cluster_label'] = Series(prediction, index = tempDF.index)

#color code assignment for prediction
tempColor = np.where(tempDF['cluster_label'] == 0, 'black', 
                     np.where( tempDF['cluster_label'] == 1, 'yellow', 
                              np.where(tempDF['cluster_label'] == 2, 'magenta', 
                                       np.where(tempDF['cluster_label'] == 3, 'cyan', 
                                                np.where(tempDF['cluster_label'] == 4, 'red', 
                                                         np.where(tempDF['cluster_label'] == 5, 'green', 
                                                                 np.where(tempDF['cluster_label'] == 6, 'blue', 'orange'))))) ))


plt.scatter(centroidsDF['X'], centroidsDF['Y'], c='black', linewidths=50, marker='x')
plt.scatter(tempDF['X'], tempDF['Y'], c=tempColor, s=50 )

# plt.gca().invert_yaxis()
plt.show()

im = plt.imread('overlayingImage.png')
implot = plt.imshow(im)

80928
30348
